In [ ]:
# Natural language processing library.
import csv, nltk, pickle
import pandas as pd
import numpy as np
from io import StringIO
from collections import Counter
from scipy import sparse

# need to use once to download nltk on your computer.
# nltk.download()

Vu qu'on a une librarie qui nous permet de faire pas mal de choses, on va:
- mettre tout en minuscule
- retirer la ponctuation
- retirer tous les nombres et caractères non alphanumériques
- les tokeniser: donc séparer les mots
- retirer les stop words (and, the, etc)
- stemmer -> avoir que la racine de chaque mots

à partir de là on aura déjà un dataset plus ou moins propre =)

In [ ]:
# Each new line will be a new element of array
with open("twitter-datasets/train_neg_full.txt", "r", encoding="utf8") as myfile:
    data_neg = myfile.readlines()
with open("twitter-datasets/train_pos_full.txt", "r", encoding="utf8") as myfile:
    data_pos = myfile.readlines()
with open("twitter-datasets/test_data.txt", "r", encoding="utf8") as myfile:
    data_test = myfile.readlines()

In [ ]:
# Transform array into Dataframe to use it easily
negative_DF = pd.DataFrame(data_neg)
positive_DF = pd.DataFrame(data_pos)
test_DF = pd.DataFrame(data_test)

In [ ]:
negative_DF.head(10)

In [ ]:
#Put everything to lowercase.
negative_DF[0] = negative_DF[0].astype(str).str.lower()
positive_DF[0] = positive_DF[0].astype(str).str.lower()
test_DF[0] = test_DF[0].astype(str).str.lower()

In [ ]:
negative_DF.head(10)

In [ ]:
# Take out user and url
negative_DF[0] = negative_DF[0].str.replace("<user>", '')
negative_DF[0] = negative_DF[0].str.replace("<url>", '')

positive_DF[0] = positive_DF[0].str.replace("<user>", '')
positive_DF[0] = positive_DF[0].str.replace("<url>", '')

test_DF[0] = test_DF[0].str.replace("<user>", '')
test_DF[0] = test_DF[0].str.replace("<url>", '')

In [ ]:
negative_DF.head(10)

In [ ]:
#take out punctuation and non alphanumerical characters
negative_DF[0] = negative_DF[0].str.replace(r'[^\w\s]', '')
positive_DF[0] = positive_DF[0].str.replace(r'[^\w\s]', '')
test_DF[0] = test_DF[0].str.replace(r'[^\w\s]', '')

In [ ]:
# take out numbers
negative_DF[0] = negative_DF[0].str.replace(r'[\d]', '')
positive_DF[0] = positive_DF[0].str.replace(r'[\d]', '')
test_DF[0] = test_DF[0].str.replace(r'[\d]', '')

In [ ]:
negative_DF.head(10)

In [ ]:
# tokenize each one.
negative_DF[0] = negative_DF[0].str.split()
positive_DF[0] = positive_DF[0].str.split()
test_DF[0] = test_DF[0].str.split()

In [ ]:
negative_DF.head(10)

In [ ]:
# we remove the stop words
lang_set = nltk.corpus.stopwords.words('english')
negative_DF[0] = negative_DF[0].apply(lambda x: [word for word in x if word not in lang_set])
positive_DF[0] = positive_DF[0].apply(lambda x: [word for word in x if word not in lang_set])
test_DF[0] = test_DF[0].apply(lambda x: [word for word in x if word not in lang_set])

In [ ]:
negative_DF.head(10)

In [ ]:
#we remove the rt (retweets)
negative_DF[0] = negative_DF[0].apply(lambda x: [word for word in x if word not in ['rt']])
positive_DF[0] = positive_DF[0].apply(lambda x: [word for word in x if word not in ['rt']])
test_DF[0] = test_DF[0].apply(lambda x: [word for word in x if word not in ['rt']])

In [ ]:
negative_DF.head(10)

J'ai remarké que stemmetize n'est pas toujours la meilleure chose à faire et tente du coup aussi de lemmetize. Ca nous rajoute de ce faite deux colonnes, pour qu'on puisse ensuite sélectionner la meilleure à utiliser.

In [ ]:
# we stemmatize (get the root only) for each word
stemmer = nltk.stem.snowball.SnowballStemmer('english')    
negative_DF['stemmed'] = negative_DF[0].apply(lambda x: [stemmer.stem(word) for word in x])
positive_DF['stemmed'] = positive_DF[0].apply(lambda x: [stemmer.stem(word) for word in x])
test_DF['stemmed'] = test_DF[0].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
negative_DF.head(10)

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
negative_DF['lemmed'] = negative_DF[0].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
positive_DF['lemmed'] = positive_DF[0].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
test_DF['lemmed'] = test_DF[0].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
negative_DF.head(10)

Les lemmetizer ne semble pas amrcher à 100% donc essayons de faire les deux (donc utiliser le stem sur le lemmetizer)

In [ ]:
negative_DF['both'] = negative_DF['lemmed'].apply(lambda x: [stemmer.stem(word) for word in x])
positive_DF['both'] = positive_DF['lemmed'].apply(lambda x: [stemmer.stem(word) for word in x])
test_DF['both'] = test_DF['lemmed'].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
negative_DF.head(10)

In [ ]:
# Getting the stemmed and lemmetized column and formatting to write it to a file
neg_final = negative_DF['both'].apply(lambda x: ' '.join(x))
pos_final = positive_DF['both'].apply(lambda x: ' '.join(x))
test_final = test_DF['both'].apply(lambda x: ' '.join(x))

In [ ]:
# Write tweets to files
with open("twitter-datasets/train_neg_proc.txt", "w", encoding="utf8") as myfile:
    neg_final.to_csv(myfile, index=False)
with open("twitter-datasets/train_pos_proc.txt", "w", encoding="utf8") as myfile:
    pos_final.to_csv(myfile, index=False)
with open("twitter-datasets/test_data_proc.txt", "w", encoding="utf8") as myfile:
    test_final.to_csv(myfile, index=False)

Maintenant, il va falloir faire ce qui est proposé dans le pdf, donc par exemple, compter les mots qui apparaissent le plus dans negatif et positif, et pourquoi pas utiliser ça ensuite dans notre algorithme pour décider si c'est positif ou négatif dans le test. =)

Quelques petites notes: 
- chaque tweet est un liste de mots
- on a quatre colonnes: les mots de base du tweet, les mots mais stemmés (racine du mot), les mots mais lemmés (idem mais se veut plus précis) et un qui fait les deux (lem puis stem)
- je pars du principe que tout est en anglais.

In [ ]:
# Get embeddings and index values
emb = np.load("outputs/embeddings.npy")
index = pd.read_csv("outputs/vocab_cut.txt", sep=" ", header=None, quoting=csv.QUOTE_NONE)
index = index[index.columns[0]].values

# Create word definition matrix
word_weights = pd.DataFrame(data=emb, index=index)

word_weights.head()

In [ ]:
# This function removes words that are not in the vocabulary from the tweets
def clean_tweets(tweets, vocab):
    tweets.apply(lambda tweet: [word for word if vocab.str.contains(word)]

<h2>FastText</h2>

In [ ]:
neg_DF = negative_DF["both"]
pos_DF = positive_DF["both"]

In [ ]:
# We want a sparse matrix that tells us for each tweet (rows) how many times a given
# word (columns) appears ("bags of words" representation)
def bags_of_words(tweets=None):
    
    # We get the vocabulary
    with open('outputs/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
        
    col = 0
    data, rows, cols = [], [], []
    
    for tweet in tweets:
        # We count word occurences in a tweet
        word_count = Counter(tweet).items()
        
        # If the word is in the vocabulary we add it in the matrix
        for word, count in word_count:
            row = vocab.get(word)
            
            if row:
                data.append(count)
                cols.append(col)
                rows.append(row)
        
        col += 1
        
    # We convert the scipy.sparse matrix to pandas.SparseSeries for ease of use
    return pd.SparseSeries.from_coo(sparse.coo_matrix((data, (rows, cols))))

In [ ]:
bags = bags_of_words(pos_DF)

In [ ]:
bags.head(20)

<h2>SVM</h2>

In [ ]:
W = pd.DataFrame([0] * 20)

In [ ]:
# Build tweet embeddings
neg_dims = pd.DataFrame(0, index=neg_DF.index, columns=word_weights.columns)
neg_dims = neg_DF.apply(lambda tweet: word_weights.loc[tweet, :].sum(axis=0))
neg_dims.head()